In [4]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

def one_hot_encode(seq):
    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

def categorical_crossentropy_2d(y_true, y_pred):
    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

paths = ('Model/mouseRRACH10000_c{}.h5'.format(x) for x in range(1, 6))
models = [load_model(y, custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for y in paths]
context = 10000

## Replace the input with your custom sequence
input_sequence = "CCAGCAATGAGCAGTCTTCTTTCTCCTGTCTCCTAGGCCTACTTAGGAGCCTACGTTTTTATCATCTTCGCTGCCTTCCTCATCTTCTTCCTAATCTTCACCTTCTTCAAAGTCCCGGAGACCAAAGGCAGGACTTTCGAGGACATTGCCCGGGCCTTCGAGGGGCAGGCGCACTCTGGAAAAGGCCCTGCCGGTGTGGAGTTGAACAGCATGCAGCCGGTCAAGGAGACCCCTGGCAACGCCTGAGCCGGGAGCACCTCCTTCACCTCCCTCCACTGTGGAAAGCCACCTCCCCTTAAGTGGGGAGACCTCATCAGGATGAACCAGGACTGCTTCTGAGTGCTGCTATTCTCACCCCACACACTCCATGAAAACTCAGCTGCACCCAATGCTGGGGTTGACCAGATCGCCAATGACTTTTAAGTGTTTGATTTGGGGGATACTTCCCTTGTAATCAGGAAAGACCAAGGAAGCCTACCTTTATATTGGGAGGGAAGGGGCCGCAGCTCCCCTTAGGTTCTAAAACCCGCTAACTAGGACAACTAGGGAGTAGGAGTAGGGTGCAGCACCGCCCCCACCACCACCTTAGTTTGCCAGGAAACAGATCTTCATACCCAGTGTGGAGGGCCCTGGGGGATTGAACAAAAGACCCCCTCCTCACTTGATACAGCTCTGCACAGCAAAGTAACTTGAGTTTTATTTATTTTATCTTCAGGTTGAATTGCATAAATATTTATTTTTTTAATTGTAATTTTACCAAATAATGAGACAGTAACGAAATTGAGGTTGGAGGGAGGTGTT"

x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
Probability = y[0, :, 1]

<ipython-input-4-144702ec750a>:18: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return map[np.fromstring(seq, np.int8) % 5]


1/1 [==============================] - 4s 4s/step


In [9]:
model = load_model('Model/mouseRRACH10000_c1.h5',custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d})

In [11]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, 4)]    0           []                               
                                                                                                  
 conv1d_40 (Conv1D)             (None, None, 32)     160         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_33 (BatchN  (None, None, 32)    128         ['conv1d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_33 (Activation)     (None, None, 32)     0           ['batch_normalization_33[0]

In [13]:
Probability.shape

(801,)

In [16]:
len(input_sequence)

801